<a href="https://colab.research.google.com/github/diem-ai/loan-prediction/blob/master/loan_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Google Drive setup on Colab**

In [1]:
from google.colab import drive

drive.mount('/content/drive')

import sys

# To add a directory with your code into a list of directories 
# which will be searched for packages
#sys.path.append('/content/drive/My Drive/Colab Notebooks')

!pip install PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)



Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
     |████████████████████████████████| 993kB 3.5MB/s 
  Stored in directory: /root/.cache/pip/wheels/fa/d2/9a/d3b6b506c2da98289e5d417215ce34b696db856643bad779f4
Successfully built PyDrive


In [2]:
import pandas as pd

file = '/content/drive/My Drive/data/lending-club-data.zip' # change it if running on local

data = pd.read_csv(file)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (19,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data.head(1)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,is_inc_v,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,not_compliant,status,inactive_loans,bad_loans,emp_length_num,grade_num,sub_grade_num,delinq_2yrs_zero,pub_rec_zero,collections_12_mths_zero,short_emp,payment_inc_ratio,final_d,last_delinq_none,last_record_none,last_major_derog_none
0,1077501,1296599,5000,5000,4975,36 months,10.65,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,20111201T000000,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/22/11 > I need to upgra...,credit_card,Computer,860xx,AZ,27.65,0.0,19850101T000000,1.0,NaN,NaN,3.0,0.0,13648,83.7,9.0,f,0.0,0.0,5861.07,5831.78,5000.0,861.07,0.0,0.0,0.0,20150101T000000,171.62,NaN,20150101T000000,0.0,NaN,1,0,Fully Paid,1,0,11,5,0.4,1.0,1.0,1.0,0,8.1435,20141201T000000,1,1,1


In [4]:
data.columns

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'is_inc_v', 'issue_d',
       'loan_status', 'pymnt_plan', 'url', 'desc', 'purpose', 'title',
       'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line',
       'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record',
       'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
       'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'policy_code', 'not_compliant', 'status', 'inactive_loans', 'bad_loans',
       'emp_length_num', 'grade_num', 'sub_gra

In [5]:
data.dtypes

id                               int64
member_id                        int64
loan_amnt                        int64
funded_amnt                      int64
funded_amnt_inv                  int64
term                            object
int_rate                       float64
installment                    float64
grade                           object
sub_grade                       object
emp_title                       object
emp_length                      object
home_ownership                  object
annual_inc                     float64
is_inc_v                        object
issue_d                         object
loan_status                     object
pymnt_plan                      object
url                             object
desc                            object
purpose                         object
title                           object
zip_code                        object
addr_state                      object
dti                            float64
delinq_2yrs              

**Exploring the target column**

The target column (label column) of the dataset that we are interested in is called bad_loans. In this column 1 means a risky (bad) loan 0 means a safe loan. we reassign the target to be:

+1 as a safe loan

-1 as a risky (bad) loan

In [6]:
data['bad_loans'].value_counts()

0    99457
1    23150
Name: bad_loans, dtype: int64

In [7]:
data['safe_loan'] =  data['bad_loans'].apply(lambda x : 1 if x==0 else -1)
data.drop(columns=['bad_loans'], axis=1)

data.head(1)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,is_inc_v,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,not_compliant,status,inactive_loans,bad_loans,emp_length_num,grade_num,sub_grade_num,delinq_2yrs_zero,pub_rec_zero,collections_12_mths_zero,short_emp,payment_inc_ratio,final_d,last_delinq_none,last_record_none,last_major_derog_none,safe_loan
0,1077501,1296599,5000,5000,4975,36 months,10.65,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,20111201T000000,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/22/11 > I need to upgra...,credit_card,Computer,860xx,AZ,27.65,0.0,19850101T000000,1.0,NaN,NaN,3.0,0.0,13648,83.7,9.0,f,0.0,0.0,5861.07,5831.78,5000.0,861.07,0.0,0.0,0.0,20150101T000000,171.62,NaN,20150101T000000,0.0,NaN,1,0,Fully Paid,1,0,11,5,0.4,1.0,1.0,1.0,0,8.1435,20141201T000000,1,1,1,1


.Now, let us explore the distribution of the column safe_loans. This gives us a sense of how many safe and risky loans are present in the dataset. Print out the percentage of safe loans and risky loans in the data frame.

In [8]:
len(data[data['safe_loan'] == 1])/len(data)*100

81.11853319957262

In [9]:
len(data[data['safe_loan'] == -1])/len(data)*100

18.881466800427383

We have:

- Around 81% safe loans
- Around 19% risky loans

It looks like most of these loans are safe loans (thankfully). But this does make our problem of identifying risky loans challenging.

**Features for the classification algorithm**

In [10]:
features = ['grade',                     # grade of the loan
            'sub_grade',                 # sub-grade of the loan
            'short_emp',                 # one year or less of employment
            'emp_length_num',            # number of years of employment
            'home_ownership',            # home_ownership status: own, mortgage or rent
            'dti',                       # debt to income ratio
            'purpose',                   # the purpose of the loan
            'term',                      # the term of the loan
            'last_delinq_none',          # has borrower had a delinquincy
            'last_major_derog_none',     # has borrower had 90 day or worse rating
            'revol_util',                # percent of available credit being used
            'total_rec_late_fee',        # total late fees received to day
           ]

target = 'safe_loan'                   # prediction target (y) (+1 means safe, -1 is risky)

# Extract the feature columns and target column
loans = data[features + [target]]

loans.head(2)

,grade,sub_grade,short_emp,emp_length_num,home_ownership,dti,purpose,term,last_delinq_none,last_major_derog_none,revol_util,total_rec_late_fee,safe_loan
0,B,B2,0,11,RENT,27.65,credit_card,36 months,1,1,83.7,0.0,1
1,C,C4,1,1,RENT,1.00,car,60 months,1,1,9.4,0.0,-1


In [11]:
categorical_cols = loans.dtypes[loans.dtypes == 'object'].index

categorical_cols

Index(['grade', 'sub_grade', 'home_ownership', 'purpose', 'term'], dtype='object')

We have 5 columns with categorical values. We have to turn those values into numeric values using one-hot encoding

In [12]:
#loans[categorical_cols]
from sklearn.preprocessing import LabelEncoder

for cate_col in categorical_cols:
  
  le = LabelEncoder()
  #new_col = cate_col + "_le"
  loans[cate_col] =  le.fit_transform(loans[cate_col])

  
  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [13]:
loans.head()

,grade,sub_grade,short_emp,emp_length_num,home_ownership,dti,purpose,term,last_delinq_none,last_major_derog_none,revol_util,total_rec_late_fee,safe_loan
0,1,6,0,11,3,27.65,1,0,1,1,83.7,0.00,1
1,2,13,1,1,3,1.00,0,1,1,1,9.4,0.00,-1
2,2,14,0,11,3,8.72,9,0,1,1,98.5,0.00,1
3,2,10,0,11,3,20.00,8,0,0,1,21.0,16.97,1
4,0,3,0,4,3,11.20,11,0,1,1,28.3,0.00,1


We shall drop categorical columns after encoding

In [14]:
"""


loans = loans.drop(columns=categorical_cols, axis=1)

loans.head()

"""

'\n\n\nloans = loans.drop(columns=categorical_cols, axis=1)\n\nloans.head()\n\n'

Encoded columns have **_le** as suffix. They look inconvenient. We shall take out **_le** at the end of column's name so that it is named as before

In [15]:
"""

loans.columns = loans.columns.str.replace('_le', '')

loans.columns

"""

"\n\nloans.columns = loans.columns.str.replace('_le', '')\n\nloans.columns\n\n"

In [16]:
safe_loans_raw = loans[loans[target] == 1]
risky_loans_raw = loans[loans[target] == -1]

print ("Number of safe loans  : {}".format(len(safe_loans_raw)))
print ("Percentage of safe loans  : %.2f  " % (len(safe_loans_raw)/len(loans)*100) )

print ("\n")
print ("Number of risky loans : {}".format(len(risky_loans_raw)))
print ("Percentage of risky loans : %.2f " % (len(risky_loans_raw)/len(loans)*100) )


Number of safe loans  : 99457
Percentage of safe loans  : 81.12  


Number of risky loans : 23150
Percentage of risky loans : 18.88 


**Sample data to balance classes**

As we explored above, our data is disproportionally full of safe loans. Let's create two datasets: one with just the safe loans (safe_loans_raw) and one with just the risky loans (risky_loans_raw). 

One way to combat class imbalance is to **undersample** the larger class until the class distribution is approximately half and half. Here, we will **undersample** the larger class (safe loans) in order to balance out our dataset. This means we are throwing away many data points

In [17]:
# Since there are fewer risky loans than safe loans, find the ratio of the sizes
# and use that percentage to undersample the safe loans.
percentage = len(risky_loans_raw)/float(len(safe_loans_raw))

risky_loans = risky_loans_raw
safe_loans = safe_loans_raw.sample(frac=percentage, random_state=1)

# Append the risky_loans with the downsampled version of safe_loans
loans_undersampling = risky_loans.append(safe_loans)

loans_undersampling.describe()

,grade,sub_grade,short_emp,emp_length_num,home_ownership,dti,purpose,term,last_delinq_none,last_major_derog_none,revol_util,total_rec_late_fee,safe_loan
count,46300.000000,46300.000000,46300.000000,46300.000000,46300.000000,46300.000000,46300.000000,46300.000000,46300.000000,46300.000000,46300.000000,46300.000000,46300.000000
mean,2.000626,12.001793,0.129546,6.345248,1.525724,16.112822,2.933909,0.252073,0.588920,0.875529,55.814679,1.292939,0.000000
std,1.414832,6.992884,0.335807,3.754578,1.441999,7.592555,2.468216,0.434208,0.492035,0.330122,25.419163,7.025748,1.000011
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000
25%,1.000000,7.000000,0.000000,3.000000,0.000000,10.500000,2.000000,0.000000,0.000000,1.000000,37.500000,0.000000,-1.000000
50%,2.000000,11.000000,0.000000,6.000000,2.000000,16.000000,2.000000,0.000000,1.000000,1.000000,58.100000,0.000000,0.000000
75%,3.000000,16.000000,0.000000,11.000000,3.000000,21.560000,3.000000,1.000000,1.000000,1.000000,76.200000,0.000000,1.000000
max,6.000000,34.000000,1.000000,11.000000,3.000000,38.600000,11.000000,1.000000,1.000000,1.000000,150.700000,208.820000,1.000000


Now that loans_data is comprised of approximately 50% safe loans and 50% risky loans.

In [18]:
loans_undersampling['safe_loan'].value_counts()

-1    23150
 1    23150
Name: safe_loan, dtype: int64

**Split data into training and validation**

We split the data into training and validation sets using an 80/20 split and specifying seed=1 so everyone gets the same results. Call the training and validation sets train_data and validation_data, respectively.

In [19]:
from sklearn.model_selection import train_test_split

train_data, validation_data =  train_test_split(loans_undersampling, test_size=0.2)

print(train_data.shape)

print(validation_data.shape)

x_train = train_data[features]
y_train = train_data[target]

x_val = validation_data[features]
y_val = validation_data[target]

(37040, 13)
(9260, 13)


**Use decision tree in scikit-learn to build a classifier**. 

- We use the default parameters at first and use gridsearch to find the best parameters later.
- Evaluate classifier with **accuracy score, precision,  recall and F1 score**



In [20]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
import numpy as np
from sklearn.metrics import classification_report

clf = DecisionTreeClassifier()
clf.fit(x_train, y_train)

y_pred = clf.predict(x_val)

print(classification_report(y_val, y_pred))


              precision    recall  f1-score   support

          -1       0.56      0.55      0.56      4686
           1       0.55      0.56      0.55      4574

    accuracy                           0.56      9260
   macro avg       0.56      0.56      0.56      9260
weighted avg       0.56      0.56      0.56      9260



- The accuracy score is not very high, just average. 
- Precision: Fraction of risky predictions that are actually risky is 0.56 meanwhile the fraction of safe predictions that are actually safe is 0.55
- Recall: Fraction of risky data predicted to be risky is 0.5 and fraction of safe data predicted to be safe is 0.5

- The model reaches the balance between precision and recall because we already applied undersample techinque before runing classifier. We will try to improve the metrics evaluation by tuning parameters with gridsearch and cross validation


In [21]:
from sklearn.model_selection import GridSearchCV
from time import time
from sklearn import tree

parameters = {'max_depth': [2, 5, 10, 20, 50, 100]
             , 'max_features' : ['auto', 'sqrt', 'log2']
             , 'class_weight' : ['balanced', None]
             , 'criterion' : ['entropy', 'gini']
             }


tune_clf_paramer = GridSearchCV(estimator=DecisionTreeClassifier()
                   , param_grid = parameters
                   , cv=5
                   , n_jobs=-1)


tune_clf_paramer.fit(x_train, y_train)



y_pred = tune_clf_paramer.predict(x_val)

print ("best estimator: {}".format(tune_clf_paramer.best_estimator_))

print('best parameters: {}'.format(tune_clf_paramer.best_params_))

print("best core: {}".format(tune_clf_paramer.best_score_))

print("classification report:")

print(classification_report(y_val, y_pred))




best estimator: DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=10,
                       max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')
best parameters: {'class_weight': None, 'criterion': 'entropy', 'max_depth': 10, 'max_features': 'auto'}
best core: 0.6265388768898488
classification report:
              precision    recall  f1-score   support

          -1       0.62      0.65      0.64      4686
           1       0.62      0.59      0.61      4574

    accuracy                           0.62      9260
   macro avg       0.62      0.62      0.62      9260
weighted avg       0.62      0.62      0.62      9260



- Another method to deal with imbalanced dataset is **ensemble learning**. It combines the predictions of several base estimators built with a given learning algorithm in order to improve generalizability / robustness over a single estimator.

- In this notebook, I try with **bagging** and **boosting** methods.

- In scikit-learn, we use **BaggingClassifier** and **AdaBoosting** as an example of bagging and boosting method, respectively.

- We will resplit train/set with unbalanced one and using the same metrics (accuracy score, precision call and F1-score) to evaluate model


1. Bagging classifier

- A Bagging classifier is an ensemble meta-estimator that fits base classifiers each on random subsets of the original dataset and then aggregate their individual predictions (either by voting or by averaging) to form a final prediction

- If we don't specify the estimator, BaggingClassifier will use Descision Tree Classifier as default

- As usual, we build baseline model with default parameters, then, optimizing hyperparameters with gridsearch

In [0]:
loans_ensemble = loans

train_data, validation_data = train_test_split(loans_ensemble, test_size=0.2, random_state=0)

x_train = train_data[features]
y_train = train_data[target]

x_val = validation_data[features]
y_val = validation_data[target]



In [23]:
from sklearn.ensemble import BaggingClassifier

bagging_clf = BaggingClassifier()

bagging_clf.fit(x_train, y_train)
y_pred = bagging_clf.predict(x_val)

print(classification_report(y_val, y_pred))



              precision    recall  f1-score   support

          -1       0.34      0.21      0.26      4577
           1       0.83      0.90      0.87     19945

    accuracy                           0.77     24522
   macro avg       0.58      0.56      0.56     24522
weighted avg       0.74      0.77      0.75     24522



In [24]:
bagging_parameters = {'bootstrap' : [True, False]
             , 'base_estimator__max_depth': [2, 5, 10, 20, 50, 100]
             , 'base_estimator__max_features' : ['auto', 'sqrt', 'log2']
             , 'base_estimator__class_weight' : ['balanced', None]
             , 'base_estimator__criterion' : ['entropy', 'gini']
             }


tune_bagging_clf = GridSearchCV(BaggingClassifier(DecisionTreeClassifier())
                   , param_grid = bagging_parameters
                   , cv=5
                   , n_jobs=-1)



print(tune_bagging_clf.estimator.get_params().keys())


tune_bagging_clf.fit(x_train, y_train)

#dict_keys(['base_estimator__class_weight', 'base_estimator__criterion', 'base_estimator__max_depth', 'base_estimator__max_features', 'base_estimator__max_leaf_nodes', 'base_estimator__min_impurity_decrease', 'base_estimator__min_impurity_split', 'base_estimator__min_samples_leaf', 'base_estimator__min_samples_split', 'base_estimator__min_weight_fraction_leaf', 'base_estimator__presort', 'base_estimator__random_state', 'base_estimator__splitter', 'base_estimator', 'bootstrap', 'bootstrap_features', 'max_features', 'max_samples', 'n_estimators', 'n_jobs', 'oob_score', 'random_state', 'verbose', 'warm_start'])



y_pred = tune_bagging_clf.predict(x_val)

print ("best estimator: {}".format(tune_bagging_clf.best_estimator_))

print('best parameters: {}'.format(tune_bagging_clf.best_params_))

print("best core: {}".format(tune_bagging_clf.best_score_))

print("classification report:")

print(classification_report(y_val, y_pred))

dict_keys(['base_estimator__class_weight', 'base_estimator__criterion', 'base_estimator__max_depth', 'base_estimator__max_features', 'base_estimator__max_leaf_nodes', 'base_estimator__min_impurity_decrease', 'base_estimator__min_impurity_split', 'base_estimator__min_samples_leaf', 'base_estimator__min_samples_split', 'base_estimator__min_weight_fraction_leaf', 'base_estimator__presort', 'base_estimator__random_state', 'base_estimator__splitter', 'base_estimator', 'bootstrap', 'bootstrap_features', 'max_features', 'max_samples', 'n_estimators', 'n_jobs', 'oob_score', 'random_state', 'verbose', 'warm_start'])
best estimator: BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None,
                                                        criterion='gini',
                                                        max_depth=10,
                                                        max_features='auto',
                                                        max_leaf_nodes=No

2. Boosting

- In boosting methods, base estimators are built sequentially and one tries to reduce the bias of the combined estimator. The motivation is to combine several weak models to produce a powerful ensemble

- This idea is implemented in **AdaBoostClassifier** in scikit-learn

- We start with baseline model and then try with more parameters

In [25]:
from sklearn.ensemble import AdaBoostClassifier

boost_clf = AdaBoostClassifier()

boost_clf.fit(x_train, y_train)

y_pred = boost_clf.predict(x_val)

print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

          -1       0.55      0.08      0.14      4577
           1       0.82      0.99      0.90     19945

    accuracy                           0.82     24522
   macro avg       0.69      0.53      0.52     24522
weighted avg       0.77      0.82      0.75     24522



In [157]:
boost_parameters = {'learning_rate' : [0.01, 0.1, 1.0]   
             , 'n_estimators' : [10, 50, 100]
             , 'base_estimator__max_depth': [2, 5, 10, 20, 50, 100]
             , 'base_estimator__max_features' : ['auto', 'sqrt', 'log2']
             , 'base_estimator__class_weight' : ['balanced', None]
             , 'base_estimator__criterion' : ['entropy', 'gini']
             }


tune_boost_clf = GridSearchCV(AdaBoostClassifier(DecisionTreeClassifier())
                   , param_grid = boost_parameters
                   , cv=5
                   , n_jobs=-1)


print(tune_boost_clf.estimator.get_params().keys())

tune_boost_clf.fit(x_train, y_train)

y_pred = tune_boost_clf.predict(x_val)

print ("best estimator: {}".format(tune_boost_clf.best_estimator_))

print('best parameters: {}'.format(tune_boost_clf.best_params_))

print("best core: {}".format(tune_boost_clf.best_score_))

print("classification report:")

print(classification_report(y_val, y_pred))

dict_keys(['algorithm', 'base_estimator__class_weight', 'base_estimator__criterion', 'base_estimator__max_depth', 'base_estimator__max_features', 'base_estimator__max_leaf_nodes', 'base_estimator__min_impurity_decrease', 'base_estimator__min_impurity_split', 'base_estimator__min_samples_leaf', 'base_estimator__min_samples_split', 'base_estimator__min_weight_fraction_leaf', 'base_estimator__presort', 'base_estimator__random_state', 'base_estimator__splitter', 'base_estimator', 'learning_rate', 'n_estimators', 'random_state'])


/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


best estimator: AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=DecisionTreeClassifier(class_weight=None,
                                                         criterion='entropy',
                                                         max_depth=5,
                                                         max_features='sqrt',
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                         min_samples_split=2,
                                                         min_weight_fraction_leaf=0.0,
                                                         presort=False,
                                                         random_state=None,
        